# Energy Mix
**Group 4**  
Jianyi Chen, Jincheng Hong, Tianyu Su, Yicheng Huang

## 1. Problem Statement

Planet 441 is a planet with natural conditions similar to those of B.C. It relies on a combination of renewable and non-renewable energy sources to meet its power and energy needs. While hydroelectricity is a significant contributor, other energy sources including coal and natural gas, as well as emerging renewable energy technologies such as wind and solar, also play a role. However, due to a special government type, the government has a fixed budget for the cost of energy production. Our goal is to develop a mathematical optimization model that minimizes pollution while ensuring that energy needs are met efficiently.

## 2. Variables and Parameters

| Parameter               | Symbol        | Unit                |
|-------------------------|--------------|--------------------|
| LCOE Cost              | c            | \$/Mwh             |
| Pollution Cost         | p            | g/MWh CO2eq        |
| Fixed Cost             | f            | \$ per MW           |
| Production Limit       | l            | MW                 |
| Renewable Indicator    | r            | Binary (0 or 1)    |
| Consumption Rate       | consumption_rate | Ratio           |
| Total Energy Demand    | D            | MW                 |
| Total Budget          | B            | \$                  |
| Carbon Tax            | carbon_tax   | \$ per tonne CO2eq  |
| Renewable Energy Target | R_min       | Ratio              |

## 3. Assumptions and Constraints

**Task:** State assumptions and constraints. 



### **Objective Function**
Minimize the total pollution cost including carbon tax:
$$
\min \sum_{i \in \text{energy sources}} \left( \frac{p_i}{10^6} \times \text{carbon\_tax} \times x_i \right)
$$

### **Constraints**
#### **1. Energy Demand Constraint**
Ensuring the total extracted energy meets demand:
$$
\sum_{i \in \text{energy sources}} y_i \geq D
$$

#### **2. Production Limits**
Each energy source must not exceed its production capacity:
$$
x_i \leq l_i, \quad \forall i \in \text{energy sources}
$$
The extracted energy depends on the consumption rate:
$$
y_i = x_i \times \text{consumption\_rate}_i, \quad \forall i \in \text{energy sources}
$$

#### **3. Budget Constraint**
The total energy production cost must not exceed the allocated budget:
$$
\sum_{i \in \text{energy sources}} c_i \times x_i \leq B
$$

#### **4. Renewable Energy Constraint**
Ensuring at least \( R_{\min} \) proportion of energy comes from renewable sources:
$$
\sum_{i \in \text{renewable sources}} y_i \geq R_{\min} \times \sum_{i \in \text{energy sources}} y_i
$$


## 4. Build Solutions

**Task:** Construct and run the linear regression model where `concrete_compressive_strength` is the output and `cement`,`water`, `course_aggregate`, and `fine_aggregate` are the inputs.

## 5. Simple Model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pulp import LpProblem, LpVariable, LpMinimize
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

In [3]:
parameters_df = pd.read_csv("energy_parameters.csv", index_col=0)
parameters_df

,LCOE Cost ($/MWh),Pollution Cost (g/kWh CO2eq),Fixed Cost ($ per MW),Production Limit (MW),Renewable Indicator,Consumption Rate
Energy Source,,,,,,
solar,86.5,37,1516000,905.264,1,0.90
wind,106.5,12,1389000,905.264,1,0.85
hydro,213.0,150,6428500,43000.000,1,0.95
coal,74.0,1000,4074000,0.000,0,0.70
natural_gas,54.0,430,1300000,452.632,0,0.80


In [4]:
energy_sources = parameters_df.index.tolist()

c = parameters_df["LCOE Cost ($/MWh)"].to_dict()
p = parameters_df["Pollution Cost (g/kWh CO2eq)"].to_dict()
f = parameters_df["Fixed Cost ($ per MW)"].to_dict()
l = parameters_df["Production Limit (MW)"].to_dict()
r = parameters_df["Renewable Indicator"].to_dict()
consumption_rate = parameters_df["Consumption Rate"].to_dict()

D = 20000 # Total energy demand (MW)
B = 9.36e10 # Total budget ($)
carbon_tax = 10  # Carbon tax ($ per tonne CO2eq)
R_min = 0.8  # Updated renewable energy target to 0.8

x = LpVariable.dicts("x", energy_sources, lowBound=0)  # Energy supply
y = LpVariable.dicts("y", energy_sources, lowBound=0)  # Energy extraction amount

model = LpProblem("Energy_Optimization", LpMinimize)

model += lpSum((p[i] / 1e6) * carbon_tax * x[i] for i in energy_sources)

# demand
model += lpSum(y[i] for i in energy_sources) >= D

# production limits
for i in energy_sources:
    model += x[i] <= l[i]
    model += y[i] == x[i] * consumption_rate[i]  # Extraction amount = Production * Consumption rate

# Constraint: budget
model += lpSum(c[i] * x[i] for i in energy_sources) <= B

# Constraint: at least R_min percent renewable energy
renewable_energy = lpSum(y[i] * r[i] for i in energy_sources)
total_energy = lpSum(y[i] for i in energy_sources)
model += renewable_energy >= R_min * total_energy

model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/28cfc9d782034e3e86c59162517ea2de-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/28cfc9d782034e3e86c59162517ea2de-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 54 RHS
At line 68 BOUNDS
At line 69 ENDATA
Problem MODEL has 13 rows, 10 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-12) rows, 4 (-6) columns and 4 (-26) elements
0  Obj 28.505664 Primal inf 1946.413 (1)
1  Obj 29.521139
Optimal - objective value 29.521139
After Postsolve, objective 29.521139, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 29.52113936 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock secon

1

In [5]:
x

{'solar': x_solar,
 'wind': x_wind,
 'hydro': x_hydro,
 'coal': x_coal,
 'natural_gas': x_natural_gas}

In [6]:
x["coal"].varValue

0.0

#### The process of finding the best value was not successful, but that does not mean that there is no viable value, just that I don't think it is the best value. Therefore, the next step will still be to try to see the quality of CO2 emissions. Let's continue.

## 5.1 maybe use slack variables help us find the optimal value
Idea: 
If the **total energy supply cannot meet demand**, a slack variable demand}}  can be introduced, modifying the constraint from:

$$
\sum y_i \geq D
$$

to:

$$
\sum y_i + s_{\text{demand}} \geq D
$$

which allows for a certain degree of energy shortage. However, to prevent the optimizer from excessively relaxing the constraint, a penalty term:

$$
\lambda_1 \times s_{\text{demand}}
$$

is added to the objective function.

## 6. Result for simple model

In [7]:
# Calculate total CO2 mass
total_co2_mass = sum(p[i] * x[i].varValue for i in energy_sources) / 1e6  # Convert from g to tonnes

all_results = []
for i in energy_sources:
    all_results.append({
        "Energy Source": i,
        "Production (MW·h)": x[i].varValue,
        "Extraction (MW·h)": y[i].varValue,
        "Pollution Cost (g CO2eq)": p[i] * x[i].varValue,
        "Carbon Cost ($)": (p[i] / 1e6) * carbon_tax * x[i].varValue,
        "LCOE Cost ($)": c[i] * x[i].varValue,
        "Total CO2 Emissions (tonnes)": total_co2_mass
    })

results_df = pd.DataFrame(all_results)
results_df.to_csv("renewable_optimization_results.csv", index=False)

print(results_df)
print(f"Optimization completed for 80% renewable energy target. Total CO2 Emissions: {total_co2_mass:.2f} tonnes.")

  Energy Source  Production (MW·h)  Extraction (MW·h)  \
0         solar            905.264           814.7376   
1          wind            905.264           769.4744   
2         hydro          19385.040         18415.7880   
3          coal              0.000             0.0000   
4   natural_gas              0.000             0.0000   

   Pollution Cost (g CO2eq)  Carbon Cost ($)  LCOE Cost ($)  \
0                 33494.768         0.334948      78305.336   
1                 10863.168         0.108632      96410.616   
2               2907756.000        29.077560    4129013.520   
3                     0.000         0.000000          0.000   
4                     0.000         0.000000          0.000   

   Total CO2 Emissions (tonnes)  
0                      2.952114  
1                      2.952114  
2                      2.952114  
3                      2.952114  
4                      2.952114  
Optimization completed for 80% renewable energy target. Total CO2 Emission

## Little complex model

In [9]:
alpha,beta = 1,1

x = LpVariable.dicts("x", range(n), lowBound=0)
model = LpProblem("Energy_Optimization", LpMinimize)

model += alpha * lpSum(c[i] * x[i] for i in range(n)) + beta * lpSum(p[i] * x[i] for i in range(n))

#energy supply exceed demand
model += lpSum(x[i] for i in range(n)) >= D

#energy source not exceed limit
for i in range(n):
    model += x[i] <= l[i]

#λ%, propostion of enewable sources
model += lpSum(x[i] * r[i] for i in range(n)) >= lambda_ * lpSum(x[i] for i in range(n))

model.solve()

NameError: name 'n' is not defined

### Optimization for energy storage

In [ ]:
x = LpVariable.dicts("x", [(i, t) for i in sources for t in time_periods], lowBound=0)  
s = LpVariable.dicts("s", time_periods, lowBound=0)  
u = LpVariable.dicts("u", time_periods, lowBound=0)


model += alpha * lpSum(c[i] * x[i, t] for i in sources for t in time_periods) + \
         beta * lpSum(p[i] * x[i, t] for i in sources for t in time_periods)

